In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
test=pd.read_csv('/kaggle/input/DontGetKicked/test.csv')
train.info()

In [ ]:
train.drop(['MMRAcquisitionAuctionAveragePrice','MMRAcquisitionAuctionCleanPrice',
                'MMRAcquisitionRetailAveragePrice','MMRAcquisitonRetailCleanPrice',
                'MMRCurrentAuctionAveragePrice','MMRCurrentAuctionCleanPrice',
                'MMRCurrentRetailAveragePrice','MMRCurrentRetailCleanPrice'],
               inplace=True,axis=1)

test.drop(['MMRAcquisitionAuctionAveragePrice','MMRAcquisitionAuctionCleanPrice',
                'MMRAcquisitionRetailAveragePrice','MMRAcquisitonRetailCleanPrice',
                'MMRCurrentAuctionAveragePrice','MMRCurrentAuctionCleanPrice',
                'MMRCurrentRetailAveragePrice','MMRCurrentRetailCleanPrice'],
               inplace=True,axis=1)

In [ ]:
for col in train.columns:
    print(col,"..................")
    print(train[f'{col}'].value_counts(),'\n')


In [ ]:
train.isnull().sum()

In [ ]:
train.drop('Trim',inplace=True,axis=1)
test.drop('Trim',inplace=True,axis=1)

train.drop('WheelTypeID',inplace=True,axis=1)
test.drop('WheelTypeID',inplace=True,axis=1)

train.drop('PurchDate',inplace=True,axis=1)
test.drop('PurchDate',inplace=True,axis=1)


train['AUCGUART'].fillna(value="AUCunknown",inplace=True)
test['AUCGUART'].fillna(value="AUCunknown",inplace=True)

train['PRIMEUNIT'].fillna(value="PRIMEUNITunknown",inplace=True)
test['PRIMEUNIT'].fillna(value="PRIMEUNITunknown",inplace=True)

train['WheelType'].fillna(value="WheelTypeunknown",inplace=True)
test['WheelType'].fillna(value="WheelTypeunknown",inplace=True)



In [ ]:
train.dropna(axis=1, how='any',inplace=True)
train.isnull().sum()

In [ ]:
test.dropna(axis=1, how='any',inplace=True)
test.isnull().sum()

In [ ]:
not_categorical=train.drop(['RefId','IsBadBuy'],axis=1).columns[train.drop(['RefId','IsBadBuy'],axis=1).dtypes!='object']

categorical=list(set(train.drop(['RefId','IsBadBuy'],axis=1).columns).difference(set(not_categorical)))

print(not_categorical)
print(categorical)

In [ ]:
train.head()

In [ ]:


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for col in not_categorical:
    train[col]=scaler.fit_transform(train[col].to_frame())
    test[col]=scaler.transform(test[col].to_frame())
    

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in categorical:
    le.fit(test[col].append(train[col]).to_frame())
    test[col]=le.transform(test[col].to_frame())
    train[col]=le.transform(train[col].to_frame())

In [ ]:
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
X=train.drop(['RefId','IsBadBuy'],axis=1)
y=train['IsBadBuy']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier(n_neighbors=11)
KNN.fit(X_train,y_train)

In [ ]:
KNN.score(X_test,y_test)

In [ ]:
predict=KNN.predict(test.drop('RefId',axis=1))

In [ ]:
Submission=pd.DataFrame(data=predict,columns=['IsBadBuy'])
Submission.head()



In [ ]:
Submission['RefId']=test['RefId']
Submission.set_index('RefId',inplace=True)

In [ ]:
Submission.to_csv('Submission.csv')